In [1]:
import pandas as pd
import numpy as np

In [2]:
training=pd.read_csv('house-prices-advanced-regression-techniques/train.csv',index_col='Id')
training=training.replace('NA ',np.nan)
d=training.shape
temp=training.isna().sum()[training.isna().sum()>=(d[0]/2)].index
na_l=training.columns[training.isna().any()].tolist() # columns with nan in them
me=training[na_l].select_dtypes("float64").median()
for item in temp:
    m=training[item].isnull().values
    training.insert((d[1]-1),("Is there a "+item+" ?"),~m)
    training[("Is there a "+item+" ?")]=training[("Is there a "+item+" ?")].astype('int64')
for item in training[na_l].select_dtypes("object"):
    if item not in temp:
        tem=training[item].value_counts()
        tem1=tem.idxmax()
        training[item]=tem1
for item in me.index:
    training[item]=training[item].replace(np.nan,me[item])
training=pd.get_dummies(training,columns=temp)
col=training.pop('SalePrice')
d=training.shape
training.insert(d[1],'SalePrice',col)
print(temp)
training.head()

Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,PoolQC_Gd,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,0,0,208500
2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,0,0,0,181500
3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,0,0,223500
4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,0,0,0,0,0,140000
5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,0,0,0,250000


In [3]:
testing=pd.read_csv('house-prices-advanced-regression-techniques/train.csv',index_col='Id')
testing=testing.replace('NA ',np.nan)
d=testing.shape
temp=testing.isna().sum()[testing.isna().sum()>=(d[0]/2)].index
na_l=testing.columns[testing.isna().any()].tolist() # columns with nan in them
me=testing[na_l].select_dtypes("float64").median()
for item in temp:
    m=testing[item].isnull().values
    testing.insert((d[1]-1),("Is there a "+item+" ?"),~m)
    testing[("Is there a "+item+" ?")]=testing[("Is there a "+item+" ?")].astype('int64')
for item in testing[na_l].select_dtypes("object"):
    if item not in temp:
        tem=testing[item].value_counts()
        tem1=tem.idxmax()
        testing[item]=tem1
for item in me.index:
    testing[item]=testing[item].replace(np.nan,me[item])
testing=pd.get_dummies(testing,columns=temp)
col=testing.pop('SalePrice')
d=testing.shape
testing.insert(d[1],'SalePrice',col)
print(temp)
testing.head()

Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,PoolQC_Gd,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,0,0,208500
2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,0,0,0,181500
3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,0,0,223500
4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,0,0,0,0,0,140000
5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,0,0,0,250000


In [4]:
train_x=training.iloc[:,:-1]
train_y=training.iloc[:,-1]
test_x=testing.iloc[:,:-1]
test_y=testing.iloc[:,-1]
c=training.corr()
c['SalePrice'].sort_values()

Is there a Fence ?         -0.172184
Fence_MnPrv                -0.140613
Alley_Grvl                 -0.139195
KitchenAbvGr               -0.135907
EnclosedPorch              -0.128578
Is there a Alley ?         -0.123611
Fence_GdWo                 -0.100047
MSSubClass                 -0.084284
OverallCond                -0.077856
Is there a MiscFeature ?   -0.072298
MiscFeature_Shed           -0.069771
Fence_MnWw                 -0.051164
MiscFeature_Othr           -0.040538
YrSold                     -0.028923
Alley_Pave                 -0.027655
LowQualFinSF               -0.025606
MiscVal                    -0.021190
BsmtHalfBath               -0.016844
BsmtFinSF2                 -0.011378
Fence_GdPrv                -0.005152
MiscFeature_Gar2           -0.004744
PoolQC_Gd                   0.012038
PoolQC_Fa                   0.016127
MiscFeature_TenC            0.022773
3SsnPorch                   0.044584
MoSold                      0.046432
PoolArea                    0.092404
I

0.7390694117518739